In [ ]:
#1.먼저 필요한 모듈들을 import 한다.

import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore')
#2. 엔비디아 그래픽 카드를 사용하고 있다면 device에 cuda가 나와야 gpu가 성공적으로 할당된 것이다.



# 이것도 드라이버 버전, 파이토치 버전, 쿠다 버전의 호환이 각 버전마다 다르기 때문에 검색을 반드시 해서 참고해야 한다.

# 성공적으로 gpu를 할당했으면 우리가 기본적으로 사용할 image size, epoch수 learning rate, batch_size, seed를 정한다.

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

CFG = {
    'IMG_SIZE':224,
    'EPOCHS':30,
    'LEARNING_RATE':5e-4,
    'BATCH_SIZE':32,
    'SEED':41
}
# 3. 딥러닝에는 랜덤으로 작업하는 것이 많기 때문에 같은 결과를 얻기 위해서 최대한 seed를 고정한다.

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정
# 4. 경로에 있는 이미지의 주소를 전부 읽어와서 all_img_list에 리스트로 저장한다.

# 이미지 주소에 label이 포함되어 있으므로 split 함수를 통해 label을 때어내서 df['label']에 저장한다.

all_img_list = glob.glob('./train/*/*')
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('\\')[1])
5. 만들어진 df의 형태는 아래와같이 파일의 경로를 나타내는 img_path와 label을 나타내는 label로 이루어져 있다.

df

img_path	label
0	./train\가구수정\0.png	가구수정
1	./train\가구수정\1.png	가구수정
2	./train\가구수정\10.png	가구수정
3	./train\가구수정\11.png	가구수정
4	./train\가구수정\2.png	가구수정
...	...	...
3452	./train\훼손\995.png	훼손
3453	./train\훼손\996.png	훼손
3454	./train\훼손\997.png	훼손
3455	./train\훼손\998.png	훼손
3456	./train\훼손\999.png	훼손
# 6.test error를 추정하기 위해 train set을 train과 validation set으로 나눈다.

#     이후 label encoding을 통해 하자 유형을 숫자에 매핑한다.

train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, stratify=df['label'], random_state=CFG['SEED'])
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])
# 7. 이미지 데이터를 다룰때는 CustomDataset이라는 class를 정의하고 사용을 하는데 여기서 중요한 것은 __getitem__이며 이미지를 한개씩 보면서 array형식으로 저장하고 이후에 정의할 transforms를 넣으면 적용이 되도록 하였다. 또한 label이 none이 아니면 image와 label을 동시에 반환하고 test셋에는 label이 없기 때문에 label이 없으면 image만 반환하게 하였다.

from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_path_list[index]

        image = Image.open(img_path)
        image = np.array(image)

        if self.transforms is not None:
            image = self.transforms(image=image)['image']

        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img_path_list)
# 8. 다음은 이미지 증강기법이다. 이 대회에 주어진 이미지는 약 1000장 정도로 적은양의 데이터 이기 때문에 좌우반전, 크기조절, 상하반전, 밝기 조절 등을 통해서 같은 이미지를 확률적으로 변형을 시켜서 다양하게 학습할 수 있게 한다.

train_transform = A.Compose([
    A.RandomResizedCrop(CFG['IMG_SIZE'], CFG['IMG_SIZE'], scale=(0.2, 0.8)),
    A.Transpose(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=20, val_shift_limit=20, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
    A.ChannelShuffle(),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    A.CoarseDropout(p=0.5),
    ToTensorV2()
])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
# 9. 우리가 사용할 모델에 대해 작성한다. 이번에는 efficientnet_b4모델을 이용해서 트레이닝을 했다.

# 그래픽 카드가 3080ti라 그래도 잘 될거라 생각했는데 vram 12gb로는 감당이 안돼서 더 큰 모델은 사용이 불가능 했다.

class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b4(pretrained=True)
        self.backbone.classifier = nn.Sequential(
            nn.Dropout(p=0.3, inplace=True),
            nn.Linear(in_features=1280, out_features=num_classes)
        )

    def forward(self, x):
        x = self.backbone(x)
        return x
# 10. 손실함수를 정의하고 모델을 훈련시킨다. 최종적으로  f1 score를 사용해 모델을 평가할 것이고

#        손실계산은 crossEntropy를 활용했다.

def train2(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)

    best_score = 0
    best_model = None

    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device)      # ADDED .type(torch.LongTensor)

            optimizer.zero_grad()

            output = model(imgs)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')

        if scheduler is not None:
            scheduler.step(_val_score)

        if best_score < _val_score:
            best_score = _val_score
            best_model = model

    return best_model
​

def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device)      # ADDED .type(torch.LongTensor)

            pred = model(imgs)

            loss = criterion(pred, labels)

            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()

            val_loss.append(loss.item())

        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')

    return _val_loss, _val_score
# 11. 모델학습에 사용할 optimizer와 scheduler를 정의했다. Adam 옵티마이저를 사용하고 '

#        ReduceLROnPlateau 스케줄러를 활용했다.

model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train2(model, optimizer, train_loader, val_loader, scheduler, device)
# 12. 학습한 최고의 가중치를 경로에 저장해서 필요할 때 꺼낼 수 있게 한다.

#        torch.cuda.empty_cache()를 통해 cuda에 남아있는 메모리를 지운다.

torch.save(infer_model, 'C:\\Users\\82107\\Downloads\\open (2)\\b00_32_30.pt')

torch.cuda.empty_cache()
# 13. 테스트 데이터를 불러오고 예측을 진행한다.

test = pd.read_csv('./test.csv')
test_dataset0 = CustomDataset(test['img_path'].values, None, resize_transform(224,'test'))
test_loader0 = DataLoader(test_dataset0, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)

            pred = model(imgs)

            preds += pred.argmax(1).detach().cpu().numpy().tolist()

    preds = le.inverse_transform(preds)
    return preds
preds0 = inference(loaded_model0, test_loader0, device)
# 14. 제출을 진행한다.

#      처음에 그냥 제출을 했더니 0점이 나와서 당황했는데 이유를 살펴보니 파일 형식이 utf-8로 고정되어야 한다고  했고 또한   inverse_transform 으로 label 인코더를 풀어서 하면 안된다는 것을 알게 되어 직접 매칭을 해주었다.

submit0 = pd.read_csv('./sample_submission.csv')
submit0['label'] = preds0


submit0.loc[submit0['label'] == '0', 'label'] = '가구수정'
submit0.loc[submit0['label'] == '1', 'label'] = '걸레받이수정'
submit0.loc[submit0['label'] == '2', 'label'] = '곰팡이'
submit0.loc[submit0['label'] == '3', 'label'] = '꼬임'
submit0.loc[submit0['label'] == '4', 'label'] = '녹오염'
submit0.loc[submit0['label'] == '5', 'label'] = '들뜸'
submit0.loc[submit0['label'] == '6', 'label'] = '면불량'
submit0.loc[submit0['label'] == '7', 'label'] = '몰딩수정'
submit0.loc[submit0['label'] == '8', 'label'] = '반점'
submit0.loc[submit0['label'] == '9', 'label'] = '석고수정'
submit0.loc[submit0['label'] == '10', 'label'] = '오염'
submit0.loc[submit0['label'] == '11', 'label'] = '오타공'
submit0.loc[submit0['label'] == '12', 'label'] = '울음'
submit0.loc[submit0['label'] == '13', 'label'] = '이음부불량'
submit0.loc[submit0['label'] == '14', 'label'] = '창틀,문틀수정'
submit0.loc[submit0['label'] == '15', 'label'] = '터짐'
submit0.loc[submit0['label'] == '16', 'label'] = '틈새과다'
submit0.loc[submit0['label'] == '17', 'label'] = '피스'
submit0.loc[submit0['label'] == '18', 'label'] = '훼손'
submit0.to_csv('./baseline_submit100.csv', index=False)